In [2]:
import pandas as pd
import ast

from ollama import chat
from ollama import ChatResponse
all_results = {}
df = pd.read_csv('nlp(cultural_pictures).csv', sep=';',encoding ='utf-8')
df["correct_answers"] = df["correct_answers"].apply(ast.literal_eval)
df["country"] = df["country"].apply(ast.literal_eval)

#df["correct_answers_2"] = df["correct_answers_2"].apply(ast.literal_eval)
df

,id,link,question,type,subcategory,country,correct_answers,correct_answers_2
0,1,https://upload.wikimedia.org/wikipedia/commons...,With what culture/nation/national minority you...,direct,attire,[Spain],[Catalonia],NaN
1,2,https://upload.wikimedia.org/wikipedia/commons...,With culture connected to what mountains attir...,direct,attire,"[Colombia, Ecuador, Peru, Chile, Bolivia]",[Andes],NaN
2,3,https://upload.wikimedia.org/wikipedia/commons...,With what particular historical event is this ...,direct,history,[Ukraine],[Orange Revolution],NaN
3,4,https://commons.wikimedia.org/wiki/Category:Al...,In honor of which historical figure was this b...,indirect,history,[Bulgaria],"[Tsar Alexander II, Alexander II]",NaN
4,5,https://commons.wikimedia.org/wiki/Category:Av...,In what city is this building located in?,direct,culture,[Sweden],[Stockholm],NaN
5,6,https://upload.wikimedia.org/wikipedia/commons...,What is the name of this traditional dish of o...,direct,food,[Poland],[pierogi],NaN
6,7,https://upload.wikimedia.org/wikipedia/commons...,What is the name of this traditional dish?,direct,food,[Marocco],"[Briouatbriwa, briwat]",NaN
7,8,https://upload.wikimedia.org/wikipedia/commons...,What is the name of this traditional Nigerian ...,direct,food,[Nigeria],[frejon],NaN
8,9,https://upload.wikimedia.org/wikipedia/commons...,What is the name of the land/country/people be...,direct,culture,[Yoruba],[Yoruba],NaN
9,10,https://upload.wikimedia.org/wikipedia/commons...,What is the name of God(s) represented on the ...,direct,religion,[Nigeria],"[Ibeji, Ibejí, Ibeyí, Jimaguas ]",NaN


In [3]:
with open("pics/1.jpg", "rb") as f:
    image_bytes = f.read()

print("Rozmiar obrazu (bytes):", len(image_bytes))
print("Pierwsze bajty:", image_bytes[:10])

Rozmiar obrazu (bytes): 103840
Pierwsze bajty: b'\xff\xd8\xff\xe0\x00\x10JFIF'


In [4]:
from ollama import chat
from ollama import ChatResponse

with open('pics/1.jpg', 'rb') as f:
    image_bytes = f.read()

response: ChatResponse = chat(
    model='qwen3-vl:2b',
    messages=[
        {
            'role': 'user',
            'content': df.iloc[0,2],
            'images': [image_bytes],
        }
    ]
)

print(response.message.content)


The photo most closely connects to the **Tatar culture** (specifically the Tatar ethnic group). Key indicators include:  
- The distinctive **red hats and red sashes**, which are hallmarks of traditional Tatar attire (particularly in the Volga region).  
- The white shirts and dark trousers, which align with common Tatar folk dress elements.  
- The group’s dynamic, celebratory posture, and the rural, natural setting (with willow trees and a wooden fence), which reflect the cultural and social context of Tatar communities.  

While the Tatars are part of the broader Turkic ethnic group, their unique cultural identity (rooted in the Volga region) is best evidenced by these visual markers in the photo.


In [5]:
import pandas as pd
import time
import os

from ollama import chat

def generate_response_ollama(prompt, image_path, model, temperature=0.1):
    with open(image_path, "rb") as f:
        image_bytes = f.read()
    print(prompt)
    response = chat(
        model=model,
        messages=[{
            "role": "user",
            "content": prompt,
            "images": [image_bytes]
        }],
        options={
            "temperature": temperature
        }
    )

    return response.message.content or ""


def check_answer(response, expected_answers):
    if not response or not expected_answers:
        return False
 
    response_lower = response.lower()

    return any(
        expected.lower() in response_lower
        for expected in expected_answers
    )
def find_image(image_dir, i):
    for ext in ("jpg", "png"):
        path = os.path.join(image_dir, f"{i}.{ext}")
        if os.path.exists(path):
            return path
    return None

def test_vision(
    models_to_test,
    prompts,
    image_dir,
    answers,
    answers2=None
):
    results = pd.DataFrame(index=range(len(prompts)), columns=models_to_test)
    result_prompts = pd.DataFrame(index=range(len(prompts)), columns=models_to_test)

    for model_name in models_to_test:
        print(f"\n{'='*60}")
        print(f"Testowanie modelu: {model_name}")
        print(f"{'='*60}")

        for i, prompt in enumerate(prompts, start=1):
            image_path = find_image(image_dir, i)
            if image_path is None:
                print(f"⚠ Brak obrazu dla indeksu {i} (jpg/png)")
                continue
            print(f"\nPrompt {i}/{len(prompts)} | Image: {image_path}")
            response = generate_response_ollama(
                prompt=prompt,
                image_path=image_path,
                model=model_name
            )
            result_prompts.loc[i-1, model_name] = response

            correct = check_answer(response, answers[i-1])
            

            results.loc[i-1, model_name] = correct
            print(f"Wynik: {'✓' if correct else '✗'}")
        result_prompts.to_csv(
            f"{model_name[:4]}_prompts_results.csv",
            encoding='utf-8-sig',
            index=False
)

        results.to_csv(
            f"{model_name[:4]}_results.csv",
            encoding='utf-8-sig',
            index=False
)
    # Podsumowanie
    print(f"\n{'='*60}")
    print("PODSUMOWANIE")
    print(f"{'='*60}")

    for model_name in models_to_test:
        correct_count = results[model_name].sum()
        total = len(results)
        accuracy = (correct_count / total * 100) if total > 0 else 0
        print(f"{model_name}: {accuracy:.1f}% ({correct_count}/{total})")

    return results, result_prompts




In [6]:
models = [
    "qwen3-vl:2b",
    "llama3.2-vision:11b"
]

results, responses = test_vision(
    models_to_test=models,
    prompts=df["question"].tolist(),
    image_dir="pics",
    answers=df['correct_answers'],
    answers2=df['correct_answers_2'])


Testowanie modelu: qwen3-vl:2b

Prompt 1/40 | Image: pics\1.jpg
With what culture/nation/national minority you connect the photo with
Wynik: ✗

Prompt 2/40 | Image: pics\2.jpg
With culture connected to what mountains attire of this man is connected to
Wynik: ✓

Prompt 3/40 | Image: pics\3.jpg
With what particular historical event is this photo connected with
Wynik: ✗

Prompt 4/40 | Image: pics\4.jpg
In honor of which historical figure was this builfing built?
Wynik: ✗

Prompt 5/40 | Image: pics\5.jpg
In what city is this building located in?
Wynik: ✗

Prompt 6/40 | Image: pics\6.jpg
What is the name of this traditional dish of one of the central-european countries?
Wynik: ✗

Prompt 7/40 | Image: pics\7.jpg
What is the name of this traditional dish?
Wynik: ✗

Prompt 8/40 | Image: pics\8.png
What is the name of this traditional Nigerian soup?
Wynik: ✗

Prompt 9/40 | Image: pics\9.png
What is the name of the land/country/people being represented with this flag?
Wynik: ✗

Prompt 10/40 | I

ResponseError: model requires more system memory (8.2 GiB) than is available (5.7 GiB) (status code: 500)

In [63]:
response.choices[0].message

ChatCompletionOutputMessage(role='assistant', content='\nLeść komentarzeznawstwo Przetestuj Tiktaka w Kalkulatorze podobieństwa tekstów! [istnieje ją Marynarczyk]: Sztuczna inteligencja - rozbiór, co to jest? Nagrywanie wideo kamerą z derektorem z GoPro militaria tableta Ulepszenie inteligencji sztucznej z wykorzystaniem AR Niech schraniafik sztucznej inteligencji - monter, nauka | Touch Draw 3D! Jaka jest sztuczna inteligencja, jaka odmiana sztucznej inteligencji, jaka sztuczna inteligencja, co to jest sztuczna inteligencja i w jaki sposób działa sztuczna', reasoning=None, tool_call_id=None, tool_calls=None)